# FUNCTIONS

In [1]:
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    total_size = sum(p.element_size() * p.numel() for p in model.parameters())
    # SIZE in GB
    print(f"TOTAL PARAMS : {total_params}")
    print(f"MODEL SIZE : {total_size/(1024**3)} GB")

    
def evaluate_model(model, loader, criterion):
    
    model.eval()
    total_loss=0
    with torch.no_grad():
        for batch in loader:

            # Your training code here
            batch = batch[0].to(device)  # Move data to the device if using GPU
            outputs = model(batch)

            recon_loss = criterion(outputs, batch).item()*len(batch)
#             regularization_loss = model.calculate_regularization_loss()
#             loss = recon_loss + regularization_loss

            #train_loss_liss.append(round(loss.item(),6))
            total_loss += round(recon_loss,6)
        
    total_loss = total_loss/len(loader.dataset)

    model.train()
    return total_loss

# IMPORTS

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset

# !nvidia-smi
import argparse, os
import numpy as np
# from himalaya.backend import set_backend
# from himalaya.ridge import RidgeCV
# from himalaya.scoring import correlation_score
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# import time
# import torch
# import torch.nn as nn
# import torch.optim as optim


/home/aditya/anaconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# python ridge.py --target c --roi ventral --subject subj01
# python ridge.py --target init_latent --roi early --subject subj01

# SETUP

In [4]:
target = 'c'
roi = 'ventral'
subject = 'subj01'
# backend = set_backend("numpy", on_error="warn")

mridir = f'../../mrifeat/{subject}/'
featdir = '../../nsdfeat/subjfeat/'
savedir = f'../..//decoded_net/{subject}/'
os.makedirs(savedir, exist_ok=True)

In [6]:
X = []
X_te = []

for croi in [roi]:
    if 'conv' in target: # We use averaged features for GAN due to large number of dimension of features
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_ave_tr.npy').astype("float32")
    else:
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_tr.npy').astype("float32")
    
    cX_te = np.load(f'{mridir}/{subject}_{croi}_betas_ave_te.npy').astype("float32")
    
    X.append(cX)
    X_te.append(cX_te)

X = np.hstack(X)
X_te = np.hstack(X_te)


Y = np.load(f'{featdir}/{subject}_each_{target}_tr.npy').astype("float32").reshape([24980,-1])
Y_te = np.load(f'{featdir}/{subject}_ave_{target}_te.npy').astype("float32").reshape([X_te.shape[0],-1])

In [7]:
print(f'Now making decoding model for... {subject}:  {roi}, {target}')
print(f'X {X.shape}, Y {Y.shape}, X_te {X_te.shape}, Y_te {Y_te.shape}')

Now making decoding model for... subj01:  ventral, c
X (24980, 7604), Y (24980, 59136), X_te (982, 7604), Y_te (982, 59136)


______
_____

## Subset small data : For Experimentation

In [ ]:
# num_samples = 500
# X_train = X[:num_samples, :].copy()
# Y_train = Y[:num_samples, :].copy()
# X_test = X_te[:num_samples//5, :].copy()
# Y_test = Y_te[:num_samples//5, :].copy()

_______
_______

# DATA

In [ ]:
# from sklearn.preprocessing import StandardScaler

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Fit the scaler on your data and transform it
# Y_total = scaler.fit_transform(Y)


In [8]:
Y_total = Y
Y_total.shape, Y_total[:,2000].mean(), Y_total[:,2000].std(), Y_total.max(), Y_total.min()

((24980, 59136), -0.37102437, 0.59869695, 33.0542, -28.098944)

In [9]:
# Split the data into training and testing sets
Y_train, Y_val = train_test_split(Y_total, test_size=0.2, random_state=42)

In [10]:
# Convert data to PyTorch tensors
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float16)
train_dataset = TensorDataset(Y_train_tensor)

Y_val_tensor = torch.tensor(Y_val, dtype=torch.float16)
val_dataset = TensorDataset(Y_val_tensor)

# Create a DataLoader
batch_size = 64  # Adjust as needed
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size , shuffle=False)


In [11]:
Y_train.shape, Y_val.shape

((19984, 59136), (4996, 59136))

In [12]:
del Y_train
del Y_val

# MODEL

In [13]:
# Define a more complex autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_size, weight_decay=1e-4):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 2500),
            nn.ReLU(),
            nn.Linear(2500, encoding_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_size, 2500),
            nn.ReLU(),
            nn.Linear(2500, input_size)
        )
        
        # Add L2 regularization to the linear layers
        self.regularization = nn.MSELoss(reduction='sum')
        self.weight_decay = weight_decay

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def calculate_regularization_loss(self):
        reg_loss = 0
        for param in self.parameters():
            reg_loss += self.regularization(param, torch.zeros_like(param))
        return self.weight_decay * reg_loss
        #return 0.0


In [14]:
# Set the input and encoding dimensions
input_size = Y_train_tensor.shape[1]  # 59136
encoding_size = 1000  # Adjust the encoding size as needed
#encoding_size = 512  # Adjust the encoding size as needed


# Create the autoencoder model
model = Autoencoder(input_size, encoding_size)
model = model.half()

In [15]:
get_model_size(model)

TOTAL PARAMS : 300745136
MODEL SIZE : 0.5601814687252045 GB


In [16]:
# torch.cuda.empty_cache()
# !nvidia-smi

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=59136, out_features=2500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2500, out_features=1000, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=1000, out_features=2500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2500, out_features=59136, bias=True)
  )
  (regularization): MSELoss()
)

In [18]:
# torch.cuda.empty_cache()
# !nvidia-smi

In [19]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# TRAINING

In [ ]:
num_epochs = 500
train_loss_liss = []
val_loss_liss = []
best_loss = float('inf')
PATIENCE = 5
loss_per=100

for epoch in range(num_epochs):

    for batch_idx, batch in enumerate(train_data_loader):
    
        # Your training code here
        batch = batch[0].to(device)  # Move data to the device if using GPU
        optimizer.zero_grad()
        outputs = model(batch)
        
        # TOTAL LOSS
        recon_loss = criterion(outputs, batch)
        regularization_loss = model.calculate_regularization_loss()
        loss = recon_loss + regularization_loss
        
        loss.backward()
        optimizer.step()
        

    # AT THE END OF EPOCH
    val_loss = evaluate_model(model=model, loader=val_data_loader, criterion=criterion)
    train_loss = evaluate_model(model=model, loader=train_data_loader, criterion=criterion)
    
    if epoch>0:
        loss_per = ((old_loss-val_loss)/old_loss)*100
    
    if val_loss < best_loss:
        
        # SAVE BEST PARAMETERS
        best_params = model.state_dict()
        best_loss = val_loss
        patience_counter=0
        
    else:
        
        patience_counter +=1
        
    
    
    
    old_loss = val_loss
    
    
    val_loss_liss.append(val_loss)
    train_loss_liss.append(train_loss)
    
    print(f"EPOCH {epoch+1} COMPLETED | TRAIN LOSS :: {train_loss}| VAL LOSS :: {val_loss}")
    
    if patience_counter >= PATIENCE: #or loss_per<.01:
        print(f"\n\n BREAKING AT EPOCH {epoch+1}")
        break

EPOCH 1 COMPLETED | TRAIN LOSS :: 0.41766929168334666| VAL LOSS :: 0.41851586729383505
EPOCH 2 COMPLETED | TRAIN LOSS :: 0.3615413815552442| VAL LOSS :: 0.3632959101281025
EPOCH 3 COMPLETED | TRAIN LOSS :: 0.32283287565052043| VAL LOSS :: 0.3251220116092874
EPOCH 4 COMPLETED | TRAIN LOSS :: 0.29149765122097676| VAL LOSS :: 0.29370078102481983
EPOCH 5 COMPLETED | TRAIN LOSS :: 0.26630151776421135| VAL LOSS :: 0.26829041353082467
EPOCH 6 COMPLETED | TRAIN LOSS :: 0.24567798789031225| VAL LOSS :: 0.24746584387510007
EPOCH 7 COMPLETED | TRAIN LOSS :: 0.22831468289631698| VAL LOSS :: 0.22988312550040035
EPOCH 8 COMPLETED | TRAIN LOSS :: 0.2135922610588472| VAL LOSS :: 0.21507088430744595
EPOCH 9 COMPLETED | TRAIN LOSS :: 0.2011265261208967| VAL LOSS :: 0.20242473298638913
EPOCH 10 COMPLETED | TRAIN LOSS :: 0.19052292589071262| VAL LOSS :: 0.19176825780624496
EPOCH 11 COMPLETED | TRAIN LOSS :: 0.18147134882906318| VAL LOSS :: 0.18260975900720575
EPOCH 12 COMPLETED | TRAIN LOSS :: 0.173555738

EPOCH 95 COMPLETED | TRAIN LOSS :: 0.09033134697758204| VAL LOSS :: 0.09154755464371499
EPOCH 96 COMPLETED | TRAIN LOSS :: 0.08995927181745397| VAL LOSS :: 0.09118403162530025
EPOCH 97 COMPLETED | TRAIN LOSS :: 0.0900339899419535| VAL LOSS :: 0.09127732005604487
EPOCH 98 COMPLETED | TRAIN LOSS :: 0.08960151511208969| VAL LOSS :: 0.09084205784627698
EPOCH 99 COMPLETED | TRAIN LOSS :: 0.0897850600980784| VAL LOSS :: 0.09105375160128104
EPOCH 100 COMPLETED | TRAIN LOSS :: 0.08960166182946355| VAL LOSS :: 0.09087597237790236
EPOCH 101 COMPLETED | TRAIN LOSS :: 0.08931070646517221| VAL LOSS :: 0.09055012610088072
EPOCH 102 COMPLETED | TRAIN LOSS :: 0.08909730314251398| VAL LOSS :: 0.09034517694155327
EPOCH 103 COMPLETED | TRAIN LOSS :: 0.0892821680844676| VAL LOSS :: 0.0905357584067254
EPOCH 104 COMPLETED | TRAIN LOSS :: 0.08899678357686146| VAL LOSS :: 0.09028076941553242
EPOCH 105 COMPLETED | TRAIN LOSS :: 0.08874345546437147| VAL LOSS :: 0.09000159107285827
EPOCH 106 COMPLETED | TRAIN LO

# PLOT LOSSES 

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of x and y values
x_values = np.array(range(len(train_loss_liss))) + 1
y_values = np.array(train_loss_liss)


# Create a scatter plot
plt.scatter(x_values, y_values, label = 'TRAIN')
plt.scatter(x_values, np.array(val_loss_liss), label = 'VAL')

plt.grid()
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.title('Autoencoder Loss vs epochs')
plt.legend()

plt.show()

# SAVE MODEL

In [ ]:
# Save the entire model (including parameters, architecture, and optimizer state)
torch.save(model, 'autoencoder_c_ventral.pth')
torch.save(best_params, 'autoencoder_trained_params_c_ventral.pth')

# LOAD MODEL

In [ ]:
# Load the entire model
model_c_ventral = torch.load('autoencoder_c_ventral.pth')

# # If you saved only the model parameters
# model = Autoencoder(1000, 59136)
# model = Autoencoder(input_size, encoding_size)
# model.load_state_dict(torch.load('autoencoder_model_params.pth'))

In [ ]:
model_c_ventral